In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import argparse

In [2]:
# Check device
def get_default_device():
    if torch.cuda.is_available(): device = torch.device('cuda')
    else: device = torch.device('cpu')
    # print(device)
    return device

In [3]:
# Data transform and augmentation
def data_trans_and_aug():
    # train_tfms = transforms.Compose([   transforms.RandomCrop(32, padding=4),
    #                                     transforms.RandomHorizontalFlip(),
    #                                     transforms.ToTensor(),
    #                                     transforms.Normalize( mean = (0.4914, 0.4822, 0.4465),
    #                                                           std = (0.247, 0.243, 0.261))])
    train_tfms = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))  # 使用單通道的正規化
    ])
    valid_tfms = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))  # 使用單通道的正規化
    ])


    # valid_tfms = transforms.Compose([   transforms.ToTensor(),
    #                                     transforms.Normalize( mean = (0.4914, 0.4822, 0.4465),
    #                                                           std = (0.247, 0.243, 0.261))])
    return train_tfms, valid_tfms

In [4]:
# Test
def Test(model, device, val_loader):
    model.eval()
    val_loss = 0
    val_acc = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            out = model(images)
            val_loss += F.cross_entropy(out, labels).item()
            pred = out.argmax(dim=1, keepdim=True)
            val_acc += pred.eq(labels.view_as(pred)).sum().item()

    val_loss /= len(val_loader.dataset)
    val_acc = (100. * val_acc) / len(val_loader.dataset)
    print('\nTest Set: Average loss: {:.4f}, Accuracy: {:.2f}%'.format(val_loss, val_acc))
    return val_acc, val_loss

In [5]:
# Train
def Train(epochs, lr, model, device, train_loader, val_loader, save_path, resume):
    # optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)

    # training setting
    best_acc = 0.
    start_epoch = 1
    results = {}
    results['train_loss'] = []
    results['train_accu'] = []
    results['valid_loss'] = []
    results['valid_accu'] = []
    # training resume
    if resume:
        assert os.path.isfile(resume)
        checkpoint = torch.load(resume)
        print('Loading checkpoint model...')
        model.load_state_dict(checkpoint['model_state_dict'])
        print('Loading checkpoint optimizer...')
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        best_acc = checkpoint['best_acc']
        start_epoch = checkpoint['epoch'] + 1
        print('Start at epoch', start_epoch, 'with the best accuracy: ' + str(best_acc) + ' %')

    # Start training
    for epoch in range(start_epoch, epochs+1):
        model.train()
        print('\n')
        epoch_loss = 0.
        train_acc = 0
        for batch_idx, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            out = model(images)
            pred = out.argmax(dim=1, keepdim=True)
            train_acc += pred.eq(labels.view_as(pred)).sum().item()
            loss = F.cross_entropy(out, labels)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
            if batch_idx % 100 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(images), len(train_loader.dataset), 100. * batch_idx / len(train_loader), loss.item()))

        train_acc = (100. * train_acc) / len(train_loader.dataset)
        results['train_loss'].append(epoch_loss/len(train_loader))
        results['train_accu'].append(train_acc)
        # Validataion
        val_acc, val_loss = Test(model, device, val_loader)
        results['valid_loss'].append(val_loss)
        results['valid_accu'].append(val_acc)
        # Save Checkpoint
        if val_acc > best_acc:
            best_acc = val_acc
            print('Saving checkpoint...')
            state = {
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'epoch': epoch,
                'best_acc': best_acc }
            if not os.path.isdir(save_path): os.mkdir(save_path)
            torch.save(state, save_path + 'checkpoint.pth')

    return results

In [6]:
import matplotlib.pyplot as plt
def plot_acc_and_lr(results):
  train_loss = results['train_loss']
  val_loss = results['valid_loss']
  plt.subplot(2,1,1)
  plt.plot(train_loss, "-o", label='train_loss')
  plt.plot(val_loss, "-x", label='val_loss')
  plt.xlabel('epoch')
  plt.ylabel('loss')
  plt.legend()
  plt.title('Loss vs. Number of epochs')

  train_acc = results['train_accu']
  val_acc = results['valid_accu']
  plt.subplot(2,1,2)
  plt.plot(train_acc, "-o", label='train_acc')
  plt.plot(val_acc, "-x", label='val_acc')
  plt.xlabel('epoch')
  plt.ylabel('accuracy')
  plt.legend()
  plt.title('Accuracy vs. Number of epochs')

  plt.tight_layout()

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
def testing_for_performance(test_dl, model, device):
  model.eval()
  batch_pred_prob = []
  batch_pred_label = []
  batch_label = []
  with torch.no_grad():
    for images, labels in test_dl:
      images, labels = images.to(device), labels.to(device)
      outputs = model(images)
      pred_prob, pred_label = F.softmax(outputs, dim=1).max(1)
      batch_pred_prob.append(pred_prob.cpu())
      batch_pred_label.append(pred_label.cpu())
      batch_label.append(labels.cpu())
    return torch.cat(batch_label).numpy(), torch.cat(batch_pred_label).numpy()

from sklearn import metrics
from sklearn.metrics import multilabel_confusion_matrix
def performance(model, test_dl, device):
  labels, preds = testing_for_performance(test_dl, model, device)
  cm = metrics.confusion_matrix(labels, preds)
  accuracy = metrics.accuracy_score(labels, preds)
  precision = metrics.precision_score(labels, preds, average='macro')
  recall = metrics.recall_score(labels, preds, average='macro')
  F1_score = metrics.f1_score(labels, preds, average='macro')

  print("Accuracy:",accuracy)
  print("Precision:",precision)
  print("Recall:",recall)
  print('F1_score:',F1_score)
  cm_plot = metrics.ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=[0,1,2,3,4,5,6,7,8,9])
  cm_plot.plot()
  plt.show()

In [ ]:
from torchvision.models import resnet18, ResNet18_Weights, resnet50
Net = resnet18(pretrained=True)

Net.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)


num_ftrs = Net.fc.in_features
Net.fc = nn.Linear(num_ftrs, 10)


for param in Net.parameters():
    param.requires_grad = False

flatten_feature = Net.fc.in_features
Net.fc = nn.Linear(flatten_feature, 10)

for name, param in Net.named_parameters():
    print(f'Parameter: {name}, Requires Grad: {param.requires_grad}')

In [ ]:
def main():
    # saving
    save_path = 'checkpoint/'

    # parser: add/modify any argument if you need
    parser = argparse.ArgumentParser()
    parser.add_argument('--epochs', type=int, default=10, help='number of epochs')
    parser.add_argument('--lr', type=float, default=1e-4, help='learning rate')
    parser.add_argument('--batch_size', type=int, default=16, help='batch size')
    parser.add_argument('--resume', default='', help='resume training from checkpoint')
    opt = parser.parse_args(
        args=[
            '--epochs', '10',
            '--lr', '1e-4',
            '--batch_size', '16',
            '--resume', ''
        ]
    )

    # check if GPU is available
    device = get_default_device()

    # transform and augmentation
    train_tfms, valid_tfms = data_trans_and_aug()

    # training / validation datasets
    train_ds = datasets.MNIST('./data', train=True, transform=train_tfms, download=True)
    valid_ds = datasets.MNIST('./data', train=False, transform=valid_tfms, download=True)

    # data loaders
    train_dl = DataLoader(train_ds, batch_size=opt.batch_size, shuffle=True, num_workers=1, pin_memory=True)
    valid_dl = DataLoader(valid_ds, batch_size=opt.batch_size, shuffle=False, num_workers=1, pin_memory=True)

    # model
    model = Net.to(device)

    # Start training
    results = Train(opt.epochs, opt.lr, model, device, train_dl, valid_dl, save_path, opt.resume)

    # Finish
    print("\nTraining Finish")

    # Analysis
    performance(model, valid_dl, device)
    plot_acc_and_lr(results)

if __name__ == '__main__':
    main()


